# importing all the necessary packages

In [1]:
import pandas as pd
import numpy as np
import nltk
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


# read csv dataset then edit

In [2]:

df = pd.read_csv("googleplaystore_user_reviews.csv", na_values="nan")
df = df.dropna(subset=['App','Translated_Review','Sentiment'], how='any')
df['Sentiment'] = df['Sentiment'].replace(['Positive'],'1')
df['Sentiment'] = df['Sentiment'].replace(['Negative'],'0')
df['Sentiment'] = df['Sentiment'].replace(['Neutral'],'1')


In [3]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# reviews are categorized as lines

In [4]:
review_lines = list()
lines = df['Translated_Review'].values.tolist()
print (len(lines))

37427


# tokenization and removing punctuation and stop words

In [5]:
for line in lines :
    tokens = word_tokenize(line)
    tokens = [w.lower() for w in tokens]
    table =str.maketrans('','',string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    review_lines.append(words)
len(review_lines)
#print(review_lines)
    

37427

# word2vec model

In [6]:
import gensim

model = gensim.models.Word2Vec(sentences=review_lines,size=100,window = 5,workers =4,min_count=1)
words = list(model.wv.vocab)
print('total word: %d' %len(words))

G:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


total word: 21481


# saving the model

In [7]:
filename = 'r.txt'
model.wv.save_word2vec_format(filename,binary=False)



# word embedding as a directory of words to vectors

In [8]:
import os
embeddings_index = {}
f = open(os.path.join('','r.txt'),encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word]=coefs
f.close()


# converting the word embedding into tokenized vector

In [9]:
tk = Tokenizer()
tk.fit_on_texts(review_lines)
sequences = tk.texts_to_sequences(review_lines)
word_index = tk.word_index
print("found %s unique tokens " % len(word_index))
review_pad = pad_sequences(sequences,maxlen=100)
sentiment = df['Sentiment'].values
print('Shape of review ', review_pad.shape)
print('shape of senti' , sentiment.shape)

found 21481 unique tokens 
Shape of review  (37427, 100)
shape of senti (37427,)


# map embeddings from the loaded word2vec model for each word 

In [10]:
 num_words = len(word_index) + 1
embedd = np.zeros((num_words,100))

for word , i in word_index.items():
    if i > num_words:
        continue
    embedd_vec = embeddings_index.get(word)
    if embedd_vec is not None:
        embedd[i] = embedd_vec     
print(num_words)

21482


# embedding matrix as input to the Embedding layer

In [11]:

from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,GRU
from keras.layers.embeddings import Embedding
from keras.initializers import constant

model = Sequential()
embedding_layer =  Embedding(num_words,100,embeddings_initializer= constant(embedd),input_length=100,trainable=False)
model.add(embedding_layer)
model.add(GRU(units=32, dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# training the sentiment classification model

In [12]:

VALIDATION_SPLIT = 0.2

indices = np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad = review_pad[indices]
sentiment = sentiment[indices]
num_validation = int (VALIDATION_SPLIT * review_pad.shape[0])

X_train_pad = review_pad[:-num_validation]
y_train = sentiment[:-num_validation]
X_test_pad = review_pad[-num_validation:]
y_test = sentiment[-num_validation:]



print('shape of X_train_pad ', X_train_pad.shape)
print('shape of y_train ', y_train.shape)

print('shape of X_test_pad ', X_test_pad.shape)
print('shape of y_train ', y_test.shape)


shape of X_train_pad  (29942, 100)
shape of y_train  (29942,)
shape of X_test_pad  (7485, 100)
shape of y_train  (7485,)


# training the classification model on train and validation test set

In [13]:
model.fit(X_train_pad,y_train,batch_size=64,epochs=10,validation_data= (X_test_pad,y_test),verbose=2)
scores = model.evaluate(X_test_pad, y_test, verbose=0)


Instructions for updating:
Use tf.cast instead.
Train on 29942 samples, validate on 7485 samples
Epoch 1/10
 - 30s - loss: 0.4379 - acc: 0.7939 - val_loss: 0.3727 - val_acc: 0.8269
Epoch 2/10
 - 30s - loss: 0.3781 - acc: 0.8257 - val_loss: 0.3555 - val_acc: 0.8362
Epoch 3/10
 - 31s - loss: 0.3603 - acc: 0.8331 - val_loss: 0.3407 - val_acc: 0.8456
Epoch 4/10
 - 31s - loss: 0.3473 - acc: 0.8427 - val_loss: 0.3310 - val_acc: 0.8488
Epoch 5/10
 - 32s - loss: 0.3410 - acc: 0.8453 - val_loss: 0.3381 - val_acc: 0.8425
Epoch 6/10
 - 32s - loss: 0.3348 - acc: 0.8494 - val_loss: 0.3196 - val_acc: 0.8572
Epoch 7/10
 - 33s - loss: 0.3291 - acc: 0.8527 - val_loss: 0.3165 - val_acc: 0.8573
Epoch 8/10
 - 33s - loss: 0.3247 - acc: 0.8551 - val_loss: 0.3140 - val_acc: 0.8619
Epoch 9/10
 - 34s - loss: 0.3221 - acc: 0.8579 - val_loss: 0.3080 - val_acc: 0.8639
Epoch 10/10
 - 35s - loss: 0.3174 - acc: 0.8601 - val_loss: 0.3077 - val_acc: 0.8637


# printing accuracy

In [14]:
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.37%


# Testing sample dataset

In [15]:
test_sample1="just loving it"
test_sample2="I hate using this button,please fix it"
test_sample3="this app is bad "


test_samples = [test_sample1,test_sample2,test_sample3]
test_samples_tokens = tk.texts_to_sequences(test_samples)

pad =pad_sequences(test_samples_tokens,maxlen=100)

model.predict(x =pad)


array([[0.9428793 ],
       [0.22566655],
       [0.04121037]], dtype=float32)